In [1]:
%pip install spacy --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
!python -m spacy download es_core_news_lg --quiet

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [8]:
import pandas as pd
import spacy

# Cargar corpus

In [9]:
file_path = r"C:\Users\karen\Documents\HumanidadesDigitales_git\BDD_Corpus\Corpus_completo_revisado.xlsx"
corpus_completo = pd.read_excel(file_path, engine='openpyxl')


In [10]:
# Formatear fechas

meses = {
    'enero': '01',
    'febrero': '02',
    'marzo': '03',
    'abril': '04',
    'mayo': '05',
    'junio': '06',
    'julio': '07',
    'agosto': '08',
    'septiembre': '09',
    'octubre': '10',
    'noviembre': '11',
    'diciembre': '12'
}

def convertir_fecha(fecha_str):
    partes = fecha_str.split(' de ')
    dia = partes[0].zfill(2)  # Asegura 2 dígitos para días 1-9
    mes = meses[partes[1]]
    año = partes[2]
    return f"{dia}/{mes}/{año}"  # Formato DD/MM/AAAA

corpus_completo['Fecha'] = corpus_completo['Fecha'].apply(convertir_fecha)
corpus_completo['Fecha'] = pd.to_datetime(corpus_completo['Fecha'], format='%d/%m/%Y', errors='coerce')

In [ ]:
corpus_completo.head()

,Diario,Autor,Fecha,Título,Texto,Vínculo,ID
0,El Espectador,Gonzalo Hernández,2018-01-01,Fajardo: para nada tibio,"La Coalición Colombia –Partido Alianza Verde, ...",https://web.archive.org/web/20180102104221/htt...,1
1,El Espectador,Eduardo Barajas Sandoval,2018-01-01,Macedonia de Norte,Las interpretaciones de la historia sirven com...,https://web.archive.org/web/20180102104221/htt...,2
2,El Espectador,Daniel Emilio Rojas Castro,2018-01-01,El nacionalismo según Vargas Llosa,La semana pasada Mario Vargas Llosa publicó un...,https://web.archive.org/web/20180102104221/htt...,3
3,El Espectador,Reinaldo Spitaletta,2018-01-01,"Tiempo sagrado, tiempo profano","Pudiera decirse, sin ser una verdad absoluta, ...",https://web.archive.org/web/20180102104221/htt...,4
4,El Espectador,Aura Lucía Mera,2018-01-01,La rebelión de los bueyes,Lo mejor del encierro de Las Ventas fueron los...,https://web.archive.org/web/20180102104221/htt...,5


# Reconocimiento entidades nombradas con spacy

In [22]:
nlp = spacy.load("es_core_news_lg", disable=["tagger", "parser"])

In [39]:
doc = nlp(corpus_completo['Texto'][0])

In [41]:
corpus_completo['Texto'][0]

'La Coalición Colombia –Partido Alianza Verde, Polo Democrático Alternativo y Compromiso Ciudadano- presentó por fin a Sergio Fajardo como su candidato único a la Presidencia. \n\nFajardo diagnostica a Colombia como un país fracturado, indignado por la corrupción, despistado por no tener una agenda de desarrollo clara luego del boom minero energético, y “desambientado” en referencia al manejo de los recursos ambientales (http://sergiofajardo.co/).\n\nPromete un liderazgo hacia la reconciliación, una lucha intensa contra la corrupción y mejor educación como mecanismo para el fomento de una sociedad innovadora y emprendedora que pueda usar eficientemente sus capacidades. Reconoce abiertamente que la desigualdad -a nivel de las personas y de las regiones- es un obstáculo grande para una Colombia incluyente.\n\nLa energía y el énfasis de sus presentaciones sobre su visión del país inspiran y al mismo tiempo lo exponen, en ciertas ocasiones, a críticas por no presentar los detalles. En su m

## Ejemplo visualización de las entidades en el texto

In [42]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [43]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Coalición Colombia ORG
Partido Alianza Verde ORG
Polo Democrático Alternativo ORG
Compromiso Ciudadano- MISC
Sergio Fajardo PER
Presidencia ORG
Fajardo PER
Colombia LOC
Reconoce abiertamente que la desigualdad -a nivel de las personas MISC
Colombia LOC
RCN Radio MISC
Jorge Enrique Robledo PER
Fajardo PER
Fajardo PER
Fajardo PER
Farc ORG
Anunció ORG
Dijo MISC
Dian MISC
Cuestionó MISC
La base programática de su coalición MISC
Fajardo PER
Nada tibio diría yo.

 MISC
Fajardo PER
Mucho trabajo pedagógico por delante para convencer a un electorado MISC
Profesor asociado de Economía MISC
Investigación de la Pontificia Universidad Javeriana ORG


## NER para todo el corpus

In [23]:
# Lista para almacenar todas las entidades encontradas
todas_entidades = []

# Iterar sobre cada fila del dataframe
for idx, doc in enumerate(nlp.pipe(corpus_completo['Texto'])):
    # Procesar el texto con spaCy
    #doc = nlp(texto)
    
    # Obtener las entidades del documento
    for ent in doc.ents:
        # Crear tupla con toda la información requerida
        entidad_info = (
            ent.text,  # Entidad
            ent.label_,  # Label
            ent.start,  # Posición inicial (token)
            ent.end,  # Posición final (token)
            corpus_completo['ID'].iloc[idx],  # ID del documento
            corpus_completo['Título'].iloc[idx],  # Título del documento
            corpus_completo['Fecha'].iloc[idx],  # Fecha del documento
            corpus_completo['Autor'].iloc[idx]  # Autor del documento
        )
        todas_entidades.append(entidad_info)

# Crear el dataframe de entidades
entidades = pd.DataFrame(todas_entidades, columns=[
    'Entidad', 'Label', 'Posición inicio', 'Posición final', 
    'ID_col', 'Título_col', 'Fecha_col', 'Autor_col'
])

# Mostrar las primeras filas del dataframe resultante
print(entidades.head())

                        Entidad Label  Posición inicio  Posición final  \
0            Coalición Colombia   ORG                1               3   
1         Partido Alianza Verde   ORG                4               7   
2  Polo Democrático Alternativo   ORG                8              11   
3         Compromiso Ciudadano-  MISC               12              14   
4                Sergio Fajardo   PER               18              20   

   ID_col                Título_col  Fecha_col          Autor_col  
0       1  Fajardo: para nada tibio 2018-01-01  Gonzalo Hernández  
1       1  Fajardo: para nada tibio 2018-01-01  Gonzalo Hernández  
2       1  Fajardo: para nada tibio 2018-01-01  Gonzalo Hernández  
3       1  Fajardo: para nada tibio 2018-01-01  Gonzalo Hernández  
4       1  Fajardo: para nada tibio 2018-01-01  Gonzalo Hernández  


In [24]:
entidades.to_excel(r'C:\Users\karen\Documents\HumanidadesDigitales_git\Códigos\Out\entidades_spacy.xlsx', index=False)

In [ ]:
entidades = pd.read_excel(r'C:\Users\karen\Documents\HumanidadesDigitales_git\Códigos\Out\entidades_spacy.xlsx')

In [26]:
entidades

,Entidad,Label,Posición inicio,Posición final,ID_col,Título_col,Fecha_col,Autor_col
0,Coalición Colombia,ORG,1,3,1,Fajardo: para nada tibio,2018-01-01,Gonzalo Hernández
1,Partido Alianza Verde,ORG,4,7,1,Fajardo: para nada tibio,2018-01-01,Gonzalo Hernández
2,Polo Democrático Alternativo,ORG,8,11,1,Fajardo: para nada tibio,2018-01-01,Gonzalo Hernández
3,Compromiso Ciudadano-,MISC,12,14,1,Fajardo: para nada tibio,2018-01-01,Gonzalo Hernández
4,Sergio Fajardo,PER,18,20,1,Fajardo: para nada tibio,2018-01-01,Gonzalo Hernández
...,...,...,...,...,...,...,...,...
418529,Sagrado Corazón,LOC,588,590,13309,Casanare y el bicentenario,2018-12-28,Julio Londoño Paredes
418530,Casanare,LOC,626,627,13309,Casanare y el bicentenario,2018-12-28,Julio Londoño Paredes
418531,Venezuela,LOC,628,629,13309,Casanare y el bicentenario,2018-12-28,Julio Londoño Paredes
418532,De todas maneras,MISC,641,644,13309,Casanare y el bicentenario,2018-12-28,Julio Londoño Paredes


In [50]:
set(entidades[entidades['Label'] == 'PER']['Entidad'])

{'Ver Foto Rey',
 'Fontaine',
 'John Allan Hobson',
 'Anand',
 'Embajadas',
 'Hernando Cardona',
 'Julio César Turbay Ayala',
 'Angelina Vélez',
 'psologías',
 'Hathor',
 'Programarlos',
 'Pippi Långstrump',
 'El Pepe Grillo',
 'Betty Carter',
 'Obró',
 'Luis Alfredo Dávila',
 'Claudia Maritza Castiblanco',
 'marrulla',
 'Hacienda de Uribe',
 'Levitsky',
 'Eugenio Oneguin',
 'John Muir',
 'Luis Medina',
 'Elijah Cummings',
 'E. Morin',
 'Juan Carlos Onetti',
 'Simplificándola',
 'Albright',
 'Liliana Cavani',
 'Helio',
 'Luis Harss',
 'Altísimo',
 'Dice Moran',
 'Pound',
 'Instagram redcomunitariatrans',
 'Kelly Preston',
 'Schröder',
 'Show de Trump',
 'Sandra Martínez',
 'Joaquín Franco Burgos',
 'Tim Ingold',
 'Criado-Pérez',
 'Consuelo Corrales',
 'Don Nicolas Abuchaibe',
 'Yalitza Aparicio',
 'Dr. Martin Luther King Jr',
 'James Lawrence King',
 'R. Dahl',
 'Pastor López',
 'general Park',
 'Clara Lopéz de ASI',
 'Doña Simona Chávez',
 'Muhammed Yildirim',
 'Conmemoró',
 'Nicolás 

# Reconocimiento de entidades con GLiNER

In [3]:
%pip install gliner --quiet

Note: you may need to restart the kernel to use updated packages.


In [14]:
import transformers
from gliner import GLiNER

model = GLiNER.from_pretrained("urchade/gliner_multi_pii-v1")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

c:\Users\karen\anaconda3\Lib\site-packages\transformers\convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [21]:
text = corpus_completo['Texto'][164]
text = corpus_completo['Texto'][4143]

labels = ["persona", "organizacion", "lugar", "obra_de_arte", "lugar_literario", "escritor", "otra"]
entities = model.predict_entities(text, labels, threshold=0.5)


for entity in entities:
    print(entity["text"], "=>", entity["label"])

Nicanor Parra => persona
antipoemas => obra_de_arte
Perón => persona
Getúlio Vargas => persona
Carlos Ibáñez => persona
Allende => persona
Pagu => persona


In [20]:
text = corpus_completo['Texto'][4143]

labels = ["persona", "organizacion", "lugar", "otra"]
entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

c:\Users\karen\anaconda3\Lib\site-packages\gliner\data_processing\processor.py:351: UserWarning: Sentence of length 601 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")


Nicanor Parra => persona
Perón => persona
Getúlio Vargas => persona
Carlos Ibáñez => persona
Estados Unidos => lugar
Allende => persona
Pagu => persona
